In [1]:
!pip install fastapi uvicorn pydantic

In [2]:
from fastapi import FastAPI, Query, HTTPException
import datetime as dt
from typing import Optional, List
from pydantic import BaseModel, Field

In [3]:
app = FastAPI()
trades_db = []

In [4]:
class TradeDetails(BaseModel):
    buySellIndicator: str = Field(description="A value of BUY for buys, SELL for sells.")
    price: float = Field(description="The price of the Trade.")
    quantity: int = Field(description="The amount of units traded.")

class Trade(BaseModel):
    asset_class: Optional[str] = Field(alias="assetClass", default=None, description="The asset class of the instrum")
    counterparty: Optional[str] = Field(default=None, description="The counterparty the trade was executed with. May")
    instrument_id: str = Field(alias="instrumentId", description="The ISIN/ID of the instrument traded. E.g. TSLA, AA")
    instrument_name: str = Field(alias="instrumentName", description="The name of the instrument traded.")
    trade_date_time: dt.datetime = Field(alias="tradeDateTime", description="The date-time the Trade was executed")
    trade_details: TradeDetails = Field(alias="tradeDetails", description="The details of the trade, i.e. price, qua")
    trade_id: str = Field(alias="tradeId", default=None, description="The unique ID of the trade")
    trader: str = Field(description="The name of the Trader")

In [5]:
@app.get("/trades", response_model=List[Trade])
async def get_trades(
        search: Optional[str] = Query(None, description="Search term to filter trades"),
        asset_class: Optional[str] = Query(None, description="Filter by asset class"),
        start: Optional[dt.datetime] = Query(None, description="Filter by minimum trade date"),
        end: Optional[dt.datetime] = Query(None, description="Filter by maximum trade date"),
        min_price: Optional[float] = Query(None, description="Filter by minimum trade price"),
        max_price: Optional[float] = Query(None, description="Filter by maximum trade price"),
        trade_type: Optional[str] = Query(None, description="Filter by trade type (BUY or SELL)")
):
    filtered_trades = trades_db

    # Search filter
    if search:
        filtered_trades = [trade for trade in filtered_trades if search.lower() in str(trade).lower()]

    # Additional filters
    if asset_class:
        filtered_trades = [trade for trade in filtered_trades if trade.asset_class == asset_class]
    if start:
        filtered_trades = [trade for trade in filtered_trades if trade.trade_date_time >= start]
    if end:
        filtered_trades = [trade for trade in filtered_trades if trade.trade_date_time <= end]
    if min_price:
        filtered_trades = [trade for trade in filtered_trades if trade.trade_details.price >= min_price]
    if max_price:
        filtered_trades = [trade for trade in filtered_trades if trade.trade_details.price <= max_price]
    if trade_type:
        filtered_trades = [trade for trade in filtered_trades if trade.trade_details.buySellIndicator == trade_type]

    return filtered_trades

In [6]:
@app.get("/trades/{trade_id}", response_model=Trade)
async def get_trade_by_id(trade_id: str):
    trade = next((trade for trade in trades_db if trade.trade_id == trade_id), None)
    if trade is None:
        raise HTTPException(status_code=404, detail="Trade not found")
    return trade

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from uvicorn import run

run(app, host="localhost", port=8000)

INFO:     Started server process [17020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:56637 - "GET / HTTP/1.1" 404 Not Found
INFO:     ::1:56638 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:56638 - "GET /openapi.json HTTP/1.1" 200 OK
